# Create a Project
Note:  This requires that you have enabled products within SageMaker Studio

![](img/enable-service-catalog-portfolio-for-studio.png)

In [1]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client(service_name='servicecatalog', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
import time
timestamp = int(time.time())

In [3]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

# pprint(search_response)

prod-j3ufw6hl7utxm


In [4]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

pprint(describe_response)

{'Budgets': [],
 'LaunchPaths': [{'Id': 'lpv2-gj4couui6s46i',
                  'Name': 'Amazon SageMaker Solutions and ML Ops products'}],
 'ProductViewSummary': {'HasDefaultPath': False,
                        'Id': 'prodview-wbmrovteqfhoy',
                        'Name': 'MLOps template for model building, training, '
                                'and deployment',
                        'Owner': 'Amazon SageMaker',
                        'ProductId': 'prod-j3ufw6hl7utxm',
                        'ShortDescription': 'This template enables you to '
                                            'easily build, train, and deploy '
                                            'machine learning models. You can '
                                            'adopt MLOps best practices and '
                                            'enable Continuous '
                                            'Integration/Continuous Deployment '
                                            'for build

In [5]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

pa-oacphmo7m2bji


In [6]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
#        'PathId': 'string',
#         'ProvisioningParameters': [
#             {
#                 'Key': 'string',
#                 'Value': 'string'
#             },
#         ]
     }
#     Tags=[
#         {
#             'Key': 'string',
#             'Value': 'string'
#         },
#     ]
)


sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

pprint(create_response)

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:231218423789:project/dsoaws-1609731195',
 'ProjectId': 'p-yoykcefmol5e',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '112',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 04 Jan 2021 03:33:49 GMT',
                                      'x-amzn-requestid': '4c1b07e6-5ea1-4ea9-8378-e40e58061547'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4c1b07e6-5ea1-4ea9-8378-e40e58061547',
                      'RetryAttempts': 0}}


In [7]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print(sagemaker_project_name_and_id)

dsoaws-1609731195-p-yoykcefmol5e


## Attach Feature Store Permssions To Service Catalog User Role 

### (used for Code Build Pipeline Executions)

In [8]:
sc_role_name='AmazonSageMakerServiceCatalogProductsUseRole'

In [9]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

231218423789


In [10]:
sc_role_arn = 'arn:aws:iam::{}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'.format(account_id)
print(sc_role_arn)

arn:aws:iam::231218423789:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [11]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': '643e8b42-5282-4c54-90ba-68a43a261f8a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '643e8b42-5282-4c54-90ba-68a43a261f8a', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 04 Jan 2021 03:34:32 GMT'}, 'RetryAttempts': 0}}


In [12]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': 'c182507c-81bc-4bd9-8442-4edad1a48a50', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c182507c-81bc-4bd9-8442-4edad1a48a50', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 04 Jan 2021 03:34:34 GMT'}, 'RetryAttempts': 0}}


# Describe Project

In [14]:
response = sm.describe_project(ProjectName=sagemaker_project_name)
print(response)

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:231218423789:project/dsoaws-1609731195', 'ProjectName': 'dsoaws-1609731195', 'ProjectId': 'p-yoykcefmol5e', 'ProjectDescription': 'dsoaws-1609731195', 'ServiceCatalogProvisioningDetails': {'ProductId': 'prod-j3ufw6hl7utxm', 'ProvisioningArtifactId': 'pa-oacphmo7m2bji'}, 'ServiceCatalogProvisionedProductDetails': {'ProvisionedProductId': 'pp-ga47znz3akd5q'}, 'ProjectStatus': 'CreateCompleted', 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:231218423789:user-profile/d-scapt1dubpfo/antje', 'UserProfileName': 'antje', 'DomainId': 'd-scapt1dubpfo'}, 'CreationTime': datetime.datetime(2021, 1, 4, 3, 33, 49, 177000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'd85098e2-fda9-4227-a62b-9feb75e31c27', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd85098e2-fda9-4227-a62b-9feb75e31c27', 'content-type': 'application/x-amz-json-1.1', 'content-length': '616', 'date': 'Mon, 04 Jan 2021 03:37:41 GMT'}, 'RetryAttempts': 0}}


In [16]:
project_status = response['ProjectStatus']
print(project_status)

CreateCompleted


# Wait for Project Create Completed

In [87]:
# ProjectStatus': 'Pending'|'CreateInProgress'|'CreateCompleted'|'CreateFailed'|
# 'DeleteInProgress'|'DeleteFailed'|'DeleteCompleted',

import time

try:

    response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = response['ProjectStatus']

    while project_status in ['Pending', 'CreateInProgress']:
        print('Creating Project... Please wait...')
        time.sleep(30)
        response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = response['ProjectStatus']
        print('Project status: {}'.format(project_status))

    if project_status == 'CreateCompleted':   
        print('Project {}'.format(project_status))

    else:
        print('Project status: {}'.format(project_status))
        raise Exception('Project not created.')
        
except Exception as e:
    print(e)

Project status: CreateCompleted


# Clone Repos

In [18]:
print(region)

us-east-1


In [40]:
!echo $region

us-east-1


In [ ]:
# git clone https://git-codecommit.$region.amazonaws.com/v1/repos/sagemaker-$sagemaker_project_name_and_id-modelbuild
# git clone https://git-codecommit.$region.amazonaws.com/v1/repos/sagemaker-$sagemaker_project_name_and_id-modeldeploy

# TODO:  Add run_pipeline.py and codebuild-buildspec.yaml
# TODO:  We have to change this to $HOME dir or something to keep it out of our workshop/ git project

In [45]:
code_commit_repo1 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo1)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modelbuild


In [56]:
local_repo1 = '/root/{}/sagemaker-{}-modelbuild'.format(sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(local_repo1)

/root/dsoaws-1609731195-p-yoykcefmol5e/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modelbuild


In [47]:
code_commit_repo2 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo2)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modeldeploy


In [57]:
local_repo2 = '/root/{}/sagemaker-{}-modeldeploy'.format(sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(local_repo2)

/root/dsoaws-1609731195-p-yoykcefmol5e/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modeldeploy


In [50]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

In [55]:
!pwd

/root/workshop/10_pipeline


In [65]:
!git clone $code_commit_repo1 $local_repo1

Cloning into '/root/dsoaws-1609731195-p-yoykcefmol5e/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modelbuild'...
remote: Counting objects: 26, done.
Unpacking objects: 100% (26/26), done.


In [66]:
!git clone $code_commit_repo2 $local_repo2

Cloning into '/root/dsoaws-1609731195-p-yoykcefmol5e/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modeldeploy'...
remote: Counting objects: 12, done.
Unpacking objects: 100% (12/12), done.


# Copy Workshop Code Into Local Project Folders

In [67]:
workshop_project_build_code='/root/workshop/10_pipeline/project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modelbuild'
print(workshop_project_build_code)

/root/workshop/10_pipeline/project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modelbuild


In [68]:
workshop_project_deploy_code='/root/workshop/10_pipeline/project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modeldeploy'
print(workshop_project_deploy_code)

/root/workshop/10_pipeline/project-dsoaws-p-ibxfjw9nuim7/sagemaker-project-dsoaws-p-ibxfjw9nuim7-modeldeploy


In [69]:
!cp -r $workshop_project_build_code/* $local_repo1/

In [70]:
!cp -r $workshop_project_deploy_code/* $local_repo2/

# Commit New Code 

In [74]:
print(local_repo1)

/root/dsoaws-1609731195-p-yoykcefmol5e/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modelbuild


In [84]:
!cd $local_repo1; git status; git add --all .; git commit -m "new code"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   codebuild-buildspec.yml
	modified:   pipelines/run_pipeline.py
	modified:   setup.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	99_Create_Sagemaker_Pipeline_BERT_Reviews_MLOps.ipynb
	pipelines/.ipynb_checkpoints/
	pipelines/dsoaws/
	sagemaker-pipelines-project-abalone-REMOVE-ME.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[master a3d5d6d] new code
 20 files changed, 5352 insertions(+), 5 deletions(-)
 create mode 100644 99_Create_Sagemaker_Pipeline_BERT_Reviews_MLOps.ipynb
 create mode 100644 pipelines/.ipynb_checkpoints/__version__-checkpoint.py
 create mode 100644 pipelines/.ipynb_checkpoints/_utils-checkpoint.py
 create mode 100644 pipelines/.ipynb_checkpoints/get_pipeli

In [85]:
!cd $local_repo2; git status; git add --all .; git commit -m "new code"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   prod-config.json
	modified:   staging-config.json

no changes added to commit (use "git add" and/or "git commit -a")
[master 0aefae8] new code
 2 files changed, 2 insertions(+), 2 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 388 bytes | 38.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609731195-p-yoykcefmol5e-modeldeploy
   c58fde3..0aefae8  master -> master


# Completed.

In [ ]:
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

In [ ]:
%store